In [0]:
pip install pyspark faker azure-storage-file-datalake

Python interpreter will be restarted.
  Using cached azure_storage_file_datalake-12.18.1-py3-none-any.whl (258 kB)
  Using cached azure_storage_blob-12.24.1-py3-none-any.whl (408 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849792 sha256=605c8cb0bc6deeff255eb8991b8706e9c4833e85ac10cad55a7b4a4640a6f33f
  Stored in directory: /root/.cache/pip/wheels/b9/0f/0a/1bf9096f5b49f278182d7fe905a82209f2090edb24a7352b72
Successfully built pyspark
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2dbb161e-e0d5-42d0-b744-9eba86692dc2
    Can't uninstall 'typing-extensions'. No files were 

In [0]:
from azure.storage.filedatalake import DataLakeServiceClient
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
from faker import Faker
from datetime import datetime
from azure.storage.blob import BlobServiceClient
import random
import os


In [0]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import random
from faker import Faker

# Initialize Spark session
spark = SparkSession.builder.appName("ComprehensiveDataSets").getOrCreate()

# Initialize Faker for generating mock data
fake = Faker()

# Define categories and formulation types for the dataset
categories = ['Foundation', 'Lipstick', 'Mascara', 'Eyeshadow', 'Blush']
formulation_types = ['Liquid', 'Powder', 'Cream', 'Gel', 'Stick']
primary_ingredients = ['Shea Butter', 'Hyaluronic Acid', 'Vitamin E', 'Collagen', 'Aloe Vera']
status_options = ['Completed', 'In Progress', 'Failed', 'Pending']

# ---------------------------------------------
# Generate Product Formulations Dataset
# ---------------------------------------------

# Create a list to store product data
product_data = []
product_ids = []  # To store unique ProductIDs for relationships

# Generate 20 unique product formulations
for _ in range(20):
    product_id = fake.uuid4()  # Unique identifier for each product
    product_ids.append(product_id)
    product = {
        "ProductID": product_id,
        "ProductName": fake.word().capitalize() + " " + random.choice(categories),
        "Category": random.choice(categories),
        "FormulationType": random.choice(formulation_types),
        "PrimaryIngredients": ', '.join(random.sample(primary_ingredients, 2)),
        "LaunchDate": fake.date_between(start_date="-2y", end_date="today").strftime("%Y-%m-%d")
    }
    product_data.append(product)

# Define schema for Product Formulations
product_schema = StructType([
    StructField("ProductID", StringType(), True),
    StructField("ProductName", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("FormulationType", StringType(), True),
    StructField("PrimaryIngredients", StringType(), True),
    StructField("LaunchDate", StringType(), True)
])

# Create Product Formulations DataFrame
product_formulations_df = spark.createDataFrame(product_data, product_schema)

# ---------------------------------------------
# Generate Manufacturing Batch Dataset
# ---------------------------------------------

# Create a list to store batch data
batch_data = []

# Generate 50 manufacturing batches
for _ in range(50):
    batch = {
        "BatchID": fake.uuid4(),
        "ProductID": random.choice(product_ids),  # Associate with a product
        "BatchDate": fake.date_between(start_date="-1y", end_date="today").strftime("%Y-%m-%d"),
        "Quantity": random.randint(100, 1000),
        "Status": random.choice(status_options)
    }
    batch_data.append(batch)

# Define schema for Manufacturing Batches
batch_schema = StructType([
    StructField("BatchID", StringType(), True),
    StructField("ProductID", StringType(), True),
    StructField("BatchDate", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Status", StringType(), True)
])

# Create Manufacturing Batches DataFrame
manufacturing_batch_df = spark.createDataFrame(batch_data, batch_schema)

# ---------------------------------------------
# Generate Customer Feedback Dataset
# ---------------------------------------------

# Create a list to store customer feedback data
customer_feedback_data = []
customer_ids = [fake.uuid4() for _ in range(30)]  # Generate 30 unique Customer IDs

# Generate 50 customer feedback entries
for _ in range(50):
    feedback = {
        "FeedbackID": fake.uuid4(),
        "ProductID": random.choice(product_ids),
        "CustomerID": random.choice(customer_ids),
        "Rating": random.randint(1, 5),
        "Comments": fake.sentence(),
        "FeedbackDate": fake.date_between(start_date="-1y", end_date="today").strftime("%Y-%m-%d")
    }
    customer_feedback_data.append(feedback)

# Define schema for Customer Feedback
customer_feedback_schema = StructType([
    StructField("FeedbackID", StringType(), True),
    StructField("ProductID", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Rating", IntegerType(), True),
    StructField("Comments", StringType(), True),
    StructField("FeedbackDate", StringType(), True)
])

# Create Customer Feedback DataFrame
customer_feedback_df = spark.createDataFrame(customer_feedback_data, customer_feedback_schema)

# ---------------------------------------------
# Generate Sales Dataset
# ---------------------------------------------

# Create a list to store sales data
sales_data = []

# Generate 50 sales transactions
for _ in range(50):
    sale = {
        "OrderID": fake.uuid4(),
        "CustomerID": random.choice(customer_ids),
        "ProductID": random.choice(product_ids),
        "Quantity": random.randint(1, 10),
        "TotalAmount": round(random.uniform(20, 500), 2),
        "OrderDate": fake.date_between(start_date="-1y", end_date="today").strftime("%Y-%m-%d")
    }
    sales_data.append(sale)

# Define schema for Sales Data
sales_schema = StructType([
    StructField("OrderID", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("ProductID", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("TotalAmount", FloatType(), True),
    StructField("OrderDate", StringType(), True)
])

# Create Sales Data DataFrame
sales_df = spark.createDataFrame(sales_data, sales_schema)

# ---------------------------------------------
# Generate Supplier Information Dataset
# ---------------------------------------------

# Create a list to store supplier data
supplier_data = []
supplier_ids = [fake.uuid4() for _ in range(10)]  # Generate 10 unique Supplier IDs

# Generate 20 supplier records
for _ in range(20):
    supplier = {
        "SupplierID": random.choice(supplier_ids),
        "SupplierName": fake.company(),
        "Material": random.choice(primary_ingredients),
        "Cost": round(random.uniform(10, 100), 2),
        "DeliveryDate": fake.date_between(start_date="-1y", end_date="today").strftime("%Y-%m-%d")
    }
    supplier_data.append(supplier)

# Define schema for Supplier Information
supplier_schema = StructType([
    StructField("SupplierID", StringType(), True),
    StructField("SupplierName", StringType(), True),
    StructField("Material", StringType(), True),
    StructField("Cost", FloatType(), True),
    StructField("DeliveryDate", StringType(), True)
])

# Create Supplier Information DataFrame
supplier_df = spark.createDataFrame(supplier_data, supplier_schema)

# ---------------------------------------------
# Save DataFrames to CSV Files
# ---------------------------------------------

# Define base path for saving files
output_path = "dbfs:/tmp/"

# Save each DataFrame to a CSV file
product_formulations_df.coalesce(1).write.csv(output_path + "product_formulations.csv", header=True, mode="overwrite")
manufacturing_batch_df.coalesce(1).write.csv(output_path + "manufacturing_batches.csv", header=True, mode="overwrite")
customer_feedback_df.coalesce(1).write.csv(output_path + "customer_feedback.csv", header=True, mode="overwrite")
sales_df.coalesce(1).write.csv(output_path + "sales_data.csv", header=True, mode="overwrite")
supplier_df.coalesce(1).write.csv(output_path + "supplier_information.csv", header=True, mode="overwrite")


In [0]:
# Import required libraries
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import os

# Azure Storage connection details (using SAS Token for secure access)
account_name = "cdmo"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T06:57:17Z&st=2025-01-28T22:57:17Z&spr=https&sig=o5zgDJJKAc%2BteYHegTuV8CSItQ2ICNrXofDiCpwaMTw%3D"   # Replace with your SAS token
container_name = "00-landing"

# Construct the BlobServiceClient URL
blob_service_url = f"https://{account_name}.blob.core.windows.net?{sas_token}"

# Function to upload a file to Azure Data Lake Storage (ADLS)
def upload_to_adls(local_path, blob_path):
    """
    Uploads a file to Azure Data Lake Storage using SAS Token authentication.

    Args:
        local_path (str): Local file path to upload.
        blob_path (str): Path in the ADLS container to upload the file to.
    """
    try:
        # Initialize BlobServiceClient using the SAS token URL
        blob_service_client = BlobServiceClient(account_url=blob_service_url)

        # Get the BlobClient for the target file in the specified container
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_path)

        # Upload the file to ADLS
        with open(local_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
        print(f"✅ Uploaded file to ADLS: {blob_path}")
    except Exception as e:
        print(f"❌ Error uploading file to ADLS: {e}")

# Define datasets and their corresponding ADLS folders
datasets = {
    "customerfeedback": customer_feedback_df,
    "manufacturebatch": manufacturing_batch_df,
    "productformula": product_formulations_df,
    "sales": sales_df,
    "supplier": supplier_df,
}

# Get the current date and time for file versioning
current_date = datetime.now().strftime("%Y%m%d")
current_time = datetime.now().strftime("%H%M%S")

# Save DataFrames to DBFS and upload to ADLS
for dataset_name, dataframe in datasets.items():
    try:
        # Define temporary paths for saving and accessing data
        dbfs_temp_dir = f"dbfs:/tmp/{dataset_name}"  # Temporary DBFS directory for Spark output
        local_temp_dir = f"/dbfs/tmp/{dataset_name}"  # Local path to access DBFS files
        os.makedirs(local_temp_dir, exist_ok=True)

        # Save the DataFrame to DBFS as a single CSV file
        dataframe.coalesce(1).write.csv(dbfs_temp_dir, header=True, mode="overwrite")

        # Locate the part file in the DBFS directory
        files = dbutils.fs.ls(dbfs_temp_dir)
        part_file = next(f.path for f in files if f.name.startswith("part-"))

        # Rename the part file to a meaningful name with date and time
        local_file_path = os.path.join(local_temp_dir, f"{dataset_name}_{current_date}_{current_time}.csv")
        dbutils.fs.cp(part_file, f"file:{local_file_path}")

        # Define the blob path in ADLS with the date folder and timestamped file name
        adls_blob_path = f"data/incoming/{dataset_name}/{dataset_name}_{current_date}_{current_time}.csv"

        # Upload the file to ADLS
        upload_to_adls(local_file_path, adls_blob_path)

        # Clean up the temporary DBFS directory
        dbutils.fs.rm(dbfs_temp_dir, recurse=True)

        print(f"✅ Dataset '{dataset_name}' successfully uploaded to ADLS as {adls_blob_path}.")
    except Exception as e:
        print(f"❌ Error processing dataset '{dataset_name}': {e}")


✅ Uploaded file to ADLS: data/incoming/customerfeedback/customerfeedback_20250128_225759.csv
✅ Dataset 'customerfeedback' successfully uploaded to ADLS as data/incoming/customerfeedback/customerfeedback_20250128_225759.csv.
✅ Uploaded file to ADLS: data/incoming/manufacturebatch/manufacturebatch_20250128_225759.csv
✅ Dataset 'manufacturebatch' successfully uploaded to ADLS as data/incoming/manufacturebatch/manufacturebatch_20250128_225759.csv.
✅ Uploaded file to ADLS: data/incoming/productformula/productformula_20250128_225759.csv
✅ Dataset 'productformula' successfully uploaded to ADLS as data/incoming/productformula/productformula_20250128_225759.csv.
✅ Uploaded file to ADLS: data/incoming/sales/sales_20250128_225759.csv
✅ Dataset 'sales' successfully uploaded to ADLS as data/incoming/sales/sales_20250128_225759.csv.
✅ Uploaded file to ADLS: data/incoming/supplier/supplier_20250128_225759.csv
✅ Dataset 'supplier' successfully uploaded to ADLS as data/incoming/supplier/supplier_202501

In [0]:
# Path to the specific file
file_path = "dbfs:/tmp/product_formulations.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Display the DataFrame
display(df)


ProductID,ProductName,Category,FormulationType,PrimaryIngredients,LaunchDate
c23ce5f9-d4bc-424c-80bb-c77aba34792c,Cause Lipstick,Eyeshadow,Stick,"Hyaluronic Acid, Aloe Vera",2023-03-11
3a0b7164-faca-448b-8fdc-d029d84de1c0,Mean Foundation,Foundation,Liquid,"Aloe Vera, Vitamin E",2024-09-21
284b1217-8440-4b2f-9a43-05e06681083f,Peace Foundation,Blush,Gel,"Aloe Vera, Hyaluronic Acid",2023-06-02
e9adae79-e492-43ec-94d7-fd8ff80553b0,Focus Blush,Blush,Powder,"Vitamin E, Aloe Vera",2024-01-11
0c724d89-48c2-4567-b359-3b541e56b488,Reveal Foundation,Foundation,Stick,"Hyaluronic Acid, Shea Butter",2024-01-02
c63a6c06-e1e4-4c5b-8506-4ce98c723651,It Mascara,Lipstick,Cream,"Hyaluronic Acid, Vitamin E",2024-02-21
6d868cb9-ce6c-4ae1-b4c2-75224ff5a6cc,Tree Eyeshadow,Eyeshadow,Liquid,"Vitamin E, Collagen",2023-12-27
edb4d3d7-25b2-4b1c-9811-ff00f44d3b61,These Foundation,Blush,Liquid,"Hyaluronic Acid, Vitamin E",2024-04-20
9153be43-d01b-43c4-a495-ddd94362edc2,She Foundation,Eyeshadow,Gel,"Aloe Vera, Collagen",2023-07-24
8b685165-e4c6-4c50-963d-faebfdeaebb9,Message Foundation,Lipstick,Powder,"Vitamin E, Collagen",2023-12-13
